In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import preprocessing
import route_comparison


pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 60)
pd.set_option('mode.chained_assignment',  None)

plt.rc('font',family='malgun gothic')

smart_card_data = pd.read_csv('./data/smart_card_data.csv')
getrouteinfoall = pd.read_csv('./data/getRouteInfoAll.csv', encoding = 'cp949', index_col = 0)
getstationbyrouteall = pd.read_csv('./data/getStationByRouteAll.csv', encoding = 'cp949', index_col = 0)
bus_sttn = pd.read_csv('./data/bus_sttn.csv')
route_curve = pd.read_csv('./data/route_curve.csv', encoding = 'cp949')

### getrouteinfoall
getrouteinfoall_df = getrouteinfoall[['ROUTE_CD','ROUTE_NO']] 

### bus_sttn
bus_sttn_df = bus_sttn[['bus_sttn_id', 'posx', 'posy']]

In [2]:
from route_comparison import data_preprocessing

smart_card_data = data_preprocessing.smart_card_preprocessing(smart_card_data)
smart_card_data.head(3)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,YSR_CNT,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,DAY
0,541463722,680,0,30300052,1,8002432,8001526,1,1250,0,2022-04-01 08:17:06,2022-04-01 08:30:41,1
1,540104832,675,0,30300083,1,8001279,8001538,1,1250,0,2022-04-01 09:25:42,2022-04-01 09:37:29,1
2,608695670,673,1,30300003,1,8001777,8007023,1,0,0,2022-04-01 19:12:53,2022-04-01 19:21:42,1


In [34]:
# 환승횟수 2,3번 제외
card_nos_to_remove = smart_card_data[smart_card_data['TRANSF_CNT'] == 3]['CARD_NO']
exdata = smart_card_data[~smart_card_data['CARD_NO'].isin(card_nos_to_remove)]
card_nos_to_remove = exdata[exdata['TRANSF_CNT'] == 2]['CARD_NO']
exdata = exdata[~exdata['CARD_NO'].isin(card_nos_to_remove)]
exdata.head(3)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,YSR_CNT,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,DAY
1,540104832,675,0,30300083,1,8001279,8001538,1,1250,0,2022-04-01 09:25:42,2022-04-01 09:37:29,1
2,608695670,673,1,30300003,1,8001777,8007023,1,0,0,2022-04-01 19:12:53,2022-04-01 19:21:42,1
3,741520439,676,0,30300043,1,8001428,8002717,1,1250,0,2022-04-01 07:32:15,2022-04-01 07:52:54,1


In [58]:
# 환승데이터를 묶기위해.. 값이 1개이면 승차,하차 정보로 엮을 수가 없음
exdata_transf = exdata[exdata['CARD_NO'].map(exdata['CARD_NO'].value_counts() > 1)]
exdata_transf.head(3)

,CARD_NO,TR_MEANS_TYPE,TRANSF_CNT,BUS_ROUTE_ID,USR_TYPE,GETON_BUS_STTN_ID,GETOFF_BUS_STTN_ID,YSR_CNT,GETON_PAY,GETOFF_PAY,GETON_DATETIME,GETOFF_DATETIME,DAY
1,540104832,675,0,30300083,1,8001279,8001538,1,1250,0,2022-04-01 09:25:42,2022-04-01 09:37:29,1
2,608695670,673,1,30300003,1,8001777,8007023,1,0,0,2022-04-01 19:12:53,2022-04-01 19:21:42,1
8,908798035,675,0,30300050,1,8005516,8002204,1,1250,0,2022-04-01 15:06:47,2022-04-01 15:19:36,1


In [59]:
exdata_transf.shape

(4776659, 13)

# 출발지가 8002904 [한남대학교,36.351395,127.42235]인 경우

In [50]:
# list(exdata_transf[exdata_transf['GETON_BUS_STTN_ID']==8002904]['CARD_NO'].values)

In [74]:
filtered_rows = []

target_card_numbers = list(exdata_transf[exdata_transf['GETON_BUS_STTN_ID'] == 8002904]['CARD_NO'].unique())

for card_number in target_card_numbers:
    filtered_rows.extend(exdata_transf[exdata_transf['CARD_NO'] == card_number].values)

filtered_dataframe = pd.DataFrame(filtered_rows, columns=exdata_transf.columns)

In [81]:
filtered_transf_rows = []
target_transf_card_numbers = list(filtered_dataframe[filtered_dataframe['TRANSF_CNT']==1]['CARD_NO'].unique())

for card_number in target_transf_card_numbers:
    filtered_transf_rows.extend(filtered_dataframe[filtered_dataframe['CARD_NO']==card_number].values)
    
filtered_transf_dataframe = pd.DataFrame(filtered_transf_rows, columns = filtered_dataframe.columns)

In [ ]:
filtered_transf_dataframe